### Estruturas Criptográficas - 2022-2023
### Grupo 7
#### TP1. Problema 2

Pretende-se criar uma cifra com autenticação de meta-dados a partir de um gerador pseudo-aleatório do tipo XOF. Este gerador, que tem como "seed" uma chave gerada por um KDF a partir de uma password fornecida, produzirá uma sequência de palavras ("outputs") de 64 bits. A mensagem será cifrada por blocos, i.e. será numa primeira fase dividida, e cada bloco será cifrado com o respetivo "output" gerado pelo gerador. Uma vez cifrada a mensagem, esta juntamente com os meta-dados serão autenticados utilizando a própria "seed" do gerador (a chave obtida pela KDF).

In [2]:
import os
from pickle import dumps
from cryptography.hazmat.primitives import hashes, hmac, padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

In [3]:
def generate_cipher_key(password):
    kdf = PBKDF2HMAC(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = os.urandom(16),
        iterations = 480000,
    )
    return kdf.derive(password)

def generator(seed, n):
    digest = hashes.Hash(hashes.SHAKE256(2**n * 8))
    digest.update(seed)
    return digest.finalize()

In [4]:
auth_key = b'accorded authentication key'

def encrypt(plaintext, outputs, ad):
    blocks = []
    pad = False
    for i in range(0, len(plaintext), 8):
        block = plaintext[i:i+8]
        # padding
        if len(block) < 8:
            pad = True
            padder = padding.PKCS7(64).padder()
            padded_block = padder.update(block)
            padded_block += padder.finalize()
            blocks.append(padded_block)
        else:
            blocks.append(block)
        
    ciphertext = bytearray()
    for block, output in zip(blocks, outputs):
        for b, o in zip(block, output):
            ciphertext.append(b ^ o)

    h = hmac.HMAC(auth_key, hashes.SHA256())
    h.update(ciphertext)
    h.update(ad)

    return (ciphertext, h.finalize(), pad)
        

def decrypt(ciphertext, outputs, signature, ad, pad):
    h = hmac.HMAC(auth_key, hashes.SHA256())
    h.update(ciphertext)
    h.update(ad)

    try:
        h.verify(signature)
    except:
        print('ERROR --- Different keys used')
    else:
        # decrypt ciphertext
        blocks = [ciphertext[i:i+8] for i in range(0, len(ciphertext), 8)]

        plaintext = bytearray()
        for block, output in zip(blocks, outputs):
            for b, o in zip(block, output):
                plaintext.append(b ^ o)

        plaintext_blocks = [plaintext[i:i+8] for i in range(0, len(plaintext), 8)]
        # check if block was padded or not
        if pad:
            unpadder = padding.PKCS7(64).unpadder()
            unpadded_block = unpadder.update(plaintext_blocks[-1])
            plaintext_blocks[-1] = unpadded_block + unpadder.finalize()
        
        message = ''.join([ba.decode() for ba in plaintext_blocks])
        return message

In [6]:
def run():

    message = input('Enter the message: ')
    print('Original message:\n' + message)
    n = int(input('Enter the n: '))
    password = input('Enter the password: ').encode('utf-8')

    cipher_key = generate_cipher_key(password)

    words = generator(cipher_key, n)
    outputs = [words[i:i+8] for i in range(0, len(words), 8)]

    ad = os.urandom(16)
    crypto = encrypt(bytes(message, 'utf-8'), outputs, ad)

    ciphertext = crypto[0]
    print("\nCiphertext:")
    print(bytes(ciphertext).decode('unicode_escape'))
    signature = crypto[1]

    pad = crypto[2]
    final_message = decrypt(ciphertext, outputs, signature, ad, pad)

    print("\nDecrypted message:")
    print(final_message)

run()

Original message:
hello there! :)

Ciphertext:
WöÍw¾-ÈtË¶Ü·ñe

Decrypted message:
hello there! :)
